<a href="https://colab.research.google.com/github/jashasmani/TWITTER-SENTIMENT-ANALYSIS-NLP-/blob/main/nlp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install kaggle

In [31]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [32]:
#API to fetch data from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [33]:
#Extracting zip file

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [34]:
#Importing the dependencies

import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords # does not meaning in sentence
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [35]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
# printing stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [37]:
#load the data from csv file to pandas

twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',header=None)

In [38]:
# checking number of row and columns

twitter_data.shape

(1600000, 6)

In [39]:
# printing 1st 5 rows of dataset
twitter_data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [40]:
# naming the columns and reading dataset

column_name=['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_name,encoding='ISO-8859-1',header=None)


In [41]:
twitter_data.shape

(1600000, 6)

In [42]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [43]:
# counting number of missing values

twitter_data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [44]:
# checking the distribution of the target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [45]:
# convert the target 4 to 1

twitter_data.replace({'target':{4:1}},inplace=True)

In [46]:
# checking the distribution of the target column
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

**Stemming**
it is process to reducing a word to rootword

In [48]:
# 0 means negative tweet
# 1 means positive tweet

port_stem = PorterStemmer()

In [49]:
def stemming(content):
  stemmed_content = re.sub('[^a-zAZ]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [50]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [51]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer ou ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat acebook text might cri result choo...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,enichan dive mani time ball anag save rest go ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [53]:
# separating data and label

X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [55]:
#Splitting the data to traing data and test data

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [56]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [57]:
# convertin the textual data to numerical data

vectorizer=TfidfVectorizer() # give some impotance to word

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [58]:
print(X_train)

  (0, 390745)	0.44095783406829225
  (0, 212910)	0.42097598581070167
  (0, 98630)	0.3764542685581735
  (0, 176546)	0.5276155164827365
  (0, 317770)	0.36080806071416366
  (0, 385576)	0.2793868316853507
  (1, 7381)	0.7616797059973436
  (1, 34291)	0.6479537217053393
  (2, 252290)	0.1704518763417772
  (2, 125882)	0.2009963441879067
  (2, 139364)	0.18906242499306244
  (2, 167534)	0.1647450863827416
  (2, 358474)	0.15228207164351906
  (2, 236442)	0.23989949342199712
  (2, 391031)	0.3319312855502254
  (2, 75348)	0.31776502137817425
  (2, 383083)	0.3309895678595494
  (2, 356359)	0.3203442867375477
  (2, 123700)	0.2875628866019491
  (2, 357063)	0.1850811959126269
  (2, 119175)	0.18903493771514152
  (2, 98630)	0.45924968528130683
  (3, 159673)	0.3711930299322745
  (3, 360348)	0.27223958996866227
  (3, 343171)	0.3969994257133483
  :	:
  (1279996, 286046)	0.22931598565461161
  (1279996, 215025)	0.24179884125298817
  (1279996, 254428)	0.19259319695665236
  (1279996, 238500)	0.3623782275934317
  (127

In [59]:
print(X_test)

  (0, 391505)	0.21462025527457143
  (0, 369472)	0.33208874561304114
  (0, 358474)	0.3083755256302057
  (0, 351143)	0.3371890309845289
  (0, 342947)	0.21684211232455605
  (0, 245046)	0.17303101419681402
  (0, 239375)	0.4366512750374633
  (0, 130193)	0.2338212484123469
  (0, 125921)	0.2477060109015193
  (0, 96277)	0.3615597783381589
  (0, 67716)	0.25999506172561987
  (0, 41028)	0.16014390304954054
  (0, 18611)	0.1650427669799792
  (1, 326320)	0.25268342663966786
  (1, 313025)	0.45872990929679475
  (1, 229271)	0.3032610819536218
  (1, 199425)	0.44667191507302906
  (1, 135485)	0.6235119668118899
  (1, 18611)	0.21323156407499835
  (2, 352538)	0.425272731114485
  (2, 229313)	0.26647549383599944
  (2, 147631)	0.5766892699004906
  (2, 84261)	0.3785138815547585
  (2, 43427)	0.39093030088009556
  (2, 27891)	0.3456358526846929
  :	:
  (319994, 201696)	0.3672442254631653
  (319994, 188127)	0.3316900577462399
  (319994, 167598)	0.3455057190000296
  (319994, 117184)	0.4549113692689784
  (319995, 367

In [60]:
# Logistic Regression

model=LogisticRegression(max_iter=1000)


In [61]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [76]:
# Model Evaluation
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [77]:
print('Accuracy on training data : ', training_data_accuracy)

Accuracy on training data :  0.80145234375


In [80]:
# Model Evaluation
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)

In [81]:
print('Accuracy on test data : ', test_data_accuracy)

Accuracy on test data :  0.769090625


**Save Tarined model**

In [102]:
import pickle

In [103]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [104]:
#loded the saved model
lodded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [93]:
X_new=X_test[200]
prediction=lodded_model.predict(X_new)
print(prediction)

[1]


In [101]:
# prompt: if i want to check new twitte is right or wrong the

def check_sentiment(tweet):
  # preprocess the tweet
  stemmed_tweet = stemming(tweet)
  # convert the tweet to a TF-IDF vector
  tweet_vector = vectorizer.transform([stemmed_tweet])
  # make a prediction
  prediction = model.predict(tweet_vector)[0]
  # return the prediction and the sentiment label
  if prediction == 0:
    sentiment = "Negative"
  else:
    sentiment = "Positive"
  return prediction, sentiment

# example usage
new_tweet = "Just had the most amazing day exploring the city! The weather was perfect and the food was incredible."
prediction, sentiment = check_sentiment(new_tweet)
print(f"Prediction: {prediction}, Sentiment: {sentiment}")


Prediction: 1, Sentiment: Positive
